In [207]:


from starlingo.Atom import Atom
from starlingo.Literal import BasicLiteral
from starlingo.Program import from_string, evaluate_forwards, Program
from starlingo.Rule import Fact, NormalRule
from starlingo.Symbol import Function, Term, IntegerConstant, StringConstant

In [208]:
program = """

a :- b, d.
b :- d.
c :- a.
d.
d :- not a.

"""

In [209]:
programs = from_string(program)
program = programs[0]
print(program)

#program base. a :- b, d. b :- d. c :- a. d. d :- not a.


In [210]:
tuple(evaluate_forwards(programs, report=True));

Answer 1: {
a
b
c
d
}
SAT 1


In [211]:
_xclingo_trace = BasicLiteral(Atom(Function('_xclingo_trace')))
preprocessed_rules = []
labels = {}
for i, rule in enumerate(program.rules):
    if rule.is_rule():
        trace_rule_head = rule.head
        if rule.is_fact():
            trace_rule_body = (BasicLiteral(Atom(Function('_xclingo_trace'))),)
        else:
            trace_rule_body = (*rule.body, _xclingo_trace)
        preprocessed_rule = NormalRule(trace_rule_head, trace_rule_body)
        labels[rule] = i
        preprocessed_rules.append(
            Fact(BasicLiteral(Atom(Function('_xclingo_label', (Function('id'), Term(IntegerConstant(i)))))))
        )
        preprocessed_rules.append(
            preprocessed_rule
        )

print('\n'.join(map(str, preprocessed_rules)))
print('-' * 10)
for rule, id in labels.items():
    print("{}:".format(id), rule)

_xclingo_label(id,0).
a :- b, d, _xclingo_trace.
_xclingo_label(id,1).
b :- d, _xclingo_trace.
_xclingo_label(id,2).
c :- a, _xclingo_trace.
_xclingo_label(id,3).
d :- _xclingo_trace.
_xclingo_label(id,4).
d :- not a, _xclingo_trace.
----------
0: a :- b, d.
1: b :- d.
2: c :- a.
3: d.
4: d :- not a.


In [212]:
preprocessed = Program(rules=preprocessed_rules)
print(preprocessed.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
a :- b, d, _xclingo_trace.
_xclingo_label(id,1).
b :- d, _xclingo_trace.
_xclingo_label(id,2).
c :- a, _xclingo_trace.
_xclingo_label(id,3).
d :- _xclingo_trace.
_xclingo_label(id,4).
d :- not a, _xclingo_trace.


In [213]:
translated_rules = []
for rule in preprocessed.rules:
    if rule.is_rule():
        if rule.is_normal_rule():
            if rule.head.atom.symbol.name.startswith('_xclingo'):
                translated_rules.append(rule)
                continue
            translated_body = []
            for literal in rule.body:
                symbol = literal.atom.symbol
                if symbol.name.startswith('_xclingo'):
                    translated_body.append(literal)
                else:
                    holds_literal = BasicLiteral(sign=literal.sign, atom=Atom(
                        Function('_xclingo_holds', (Function(symbol.name), Function(arguments=symbol.arguments)))))
                    translated_body.append(holds_literal)
            raw_rule = NormalRule(rule.head, rule.body[:-1])
            if not raw_rule.body:
                raw_rule = raw_rule.as_fact()
            label = labels[raw_rule]
            symbol = rule.head.atom.symbol
            translated_head = BasicLiteral(Atom(Function('_xclingo_fires', (
            Term(IntegerConstant(label)), symbol.name, Function(arguments=symbol.arguments)))))
            translated_rules.append(NormalRule(translated_head, translated_body))

translated = Program(rules=translated_rules)
print(translated.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
_xclingo_fires(0,a,()) :- _xclingo_holds(b,()), _xclingo_holds(d,()), _xclingo_trace.
_xclingo_label(id,1).
_xclingo_fires(1,b,()) :- _xclingo_holds(d,()), _xclingo_trace.
_xclingo_label(id,2).
_xclingo_fires(2,c,()) :- _xclingo_holds(a,()), _xclingo_trace.
_xclingo_label(id,3).
_xclingo_fires(3,d,()) :- _xclingo_trace.
_xclingo_label(id,4).
_xclingo_fires(4,d,()) :- not _xclingo_holds(a,()), _xclingo_trace.


In [214]:
repositioned_rules = []
for rule in translated.rules:
    if rule.is_rule():
        if rule.is_normal_rule() and not rule.is_fact():
            if _xclingo_trace in rule.body:
                head = rule.head
                body = rule.body[:-1]

                fires_rule = NormalRule(head, body)
                support_head = BasicLiteral(Atom(Function('_xclingo_holds', head.atom.symbol.arguments[1:])))
                support_body = (head,)
                support_rule = NormalRule(support_head, support_body)
                trace_head = BasicLiteral(Atom(Function('_xclingo_trace', (
                head.atom.symbol.arguments[1], head.atom.symbol.arguments[2], Term(StringConstant(""))))))
                trace_body = support_body
                trace_rule = NormalRule(trace_head, trace_body)
                repositioned_rules.append(fires_rule)
                repositioned_rules.append(support_rule)
                repositioned_rules.append(trace_rule)

        else:
            repositioned_rules.append(rule)

repositioned = Program(rules=repositioned_rules)
print(repositioned.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
_xclingo_fires(0,a,()) :- _xclingo_holds(b,()), _xclingo_holds(d,()).
_xclingo_holds(a,()) :- _xclingo_fires(0,a,()).
_xclingo_trace(a,(),"") :- _xclingo_fires(0,a,()).
_xclingo_label(id,1).
_xclingo_fires(1,b,()) :- _xclingo_holds(d,()).
_xclingo_holds(b,()) :- _xclingo_fires(1,b,()).
_xclingo_trace(b,(),"") :- _xclingo_fires(1,b,()).
_xclingo_label(id,2).
_xclingo_fires(2,c,()) :- _xclingo_holds(a,()).
_xclingo_holds(c,()) :- _xclingo_fires(2,c,()).
_xclingo_trace(c,(),"") :- _xclingo_fires(2,c,()).
_xclingo_label(id,3).
_xclingo_fires(3,d,()) :- .
_xclingo_holds(d,()) :- _xclingo_fires(3,d,()).
_xclingo_trace(d,(),"") :- _xclingo_fires(3,d,()).
_xclingo_label(id,4).
_xclingo_fires(4,d,()) :- not _xclingo_holds(a,()).
_xclingo_holds(d,()) :- _xclingo_fires(4,d,()).
_xclingo_trace(d,(),"") :- _xclingo_fires(4,d,()).


In [215]:
models = tuple(evaluate_forwards([repositioned], report=True))

Answer 1: {
_xclingo_holds(a,())
_xclingo_holds(b,())
_xclingo_holds(c,())
_xclingo_holds(d,())
_xclingo_label(id,0)
_xclingo_label(id,1)
_xclingo_label(id,2)
_xclingo_label(id,3)
_xclingo_label(id,4)
_xclingo_fires(0,a,())
_xclingo_fires(1,b,())
_xclingo_fires(2,c,())
_xclingo_fires(3,d,())
_xclingo_trace(a,(),"")
_xclingo_trace(b,(),"")
_xclingo_trace(c,(),"")
_xclingo_trace(d,(),"")
}
SAT 1


In [216]:
model = models[0]

In [217]:
causes_table = {}
inverted_labels = {}
to_explain = []
for a in model:
    if a.symbol.name.startswith('_xclingo_holds'):
        explain = Function(a.symbol.arguments[0].name, a.symbol.arguments[1].arguments)
        fires = []
        for b in model:
            if b.symbol.name.startswith('_xclingo_fires'):
                if a.symbol.arguments[0] == b.symbol.arguments[1] and a.symbol.arguments[1] == b.symbol.arguments[2]:
                    rule_id = b.symbol.arguments[0].constant.number
                    if rule_id not in inverted_labels:
                        for rule, label in labels.items():
                            if label == rule_id:
                                inverted_labels[rule_id] = rule
                                break
                    fires.append(rule_id)

        causes_table[explain] = fires
    elif a.symbol.name.startswith('_xclingo_trace'):
        explain = Function(a.symbol.arguments[0].name, a.symbol.arguments[1].arguments)
        to_explain.append(explain)

for fun, causes in causes_table.items():
    print("{}:".format(fun), causes)

a: [0]
b: [1]
c: [2]
d: [3]


In [218]:
for label, rule in inverted_labels.items():
    print("{}: ".format(label), rule)

0:  a :- b, d.
1:  b :- d.
2:  c :- a.
3:  d.


In [219]:
print('\n'.join(map(str, to_explain)))

a
b
c
d


In [220]:
for explain in to_explain:
    print('*')
    stack = [(explain, 0)]
    while stack:
        holds, indent = stack.pop()
        print(' ' * indent + '|')
        print(' ' * indent + '++-', end=' ')
        print(holds)
        for cause in causes_table[holds]:
            rule = inverted_labels[cause]
            if rule.is_normal_rule() and not rule.is_fact():
                search = tuple((lit.atom.symbol,indent+1) for lit in rule.body if lit.is_pos())
                stack.extend(search)


*
|
++- a
 |
 ++- d
 |
 ++- b
  |
  ++- d
*
|
++- b
 |
 ++- d
*
|
++- c
 |
 ++- a
  |
  ++- d
  |
  ++- b
   |
   ++- d
*
|
++- d
